In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn import tree, svm, naive_bayes,neighbors
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from imblearn.over_sampling import SMOTE, ADASYN, SVMSMOTE
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc, precision_recall_curve  ###计算roc和auc
from imblearn.over_sampling import RandomOverSampler
from imblearn.ensemble import BalancedBaggingClassifier

In [2]:
data_path_list = [
    'DR0008_activity_accumulator_2016_09.csv',
    'DR0008_activity_accumulator_2016-10.csv',
    'DR0008_activity_accumulator_2016-11.csv',
    'DR0008_activity_accumulator_2016-12.csv',
    'DR0008_activity_accumulator_2017-02.csv',
    'DR0008_activity_accumulator_2017-03.csv',
    'DR0008_activity_accumulator_2017-04.csv',
    'DR0008_activity_accumulator_2017-05.csv'
]

se1_at_risks = pd.read_csv('Std_list_atRist_2016_se1.csv')
se2_at_risks = pd.read_csv('Std_list_atRist_2016_se2.csv')

In [21]:
def int_handle_cnt(internel_handle_list, df_int_handle, name, PRE_FIX):
    df_temp = df_int_handle[df_int_handle['internal_handle'].isin(internel_handle_list)]
    df_temp = df_temp.groupby(['De-id']).count().reset_index('De-id')
    df_temp.columns = ['De-id', PRE_FIX + name]
    return df_temp


def extract_functions(df, PRE_FIX):
    df_t = df[(df['event_type']=='PAGE_ACCESS') |
              (df['event_type']=='COURSE_ACCESS') |
              (df['event_type']=='LOGIN_ATTEMPT') |
              (df['event_type']=='SESSION_TIMEOUT') |
              (df['event_type']=='LOGOUT')]


    df_evt = df_t[['De-id', 'event_type']]
    df_login = df_evt[df_evt['event_type'] == 'LOGIN_ATTEMPT'].groupby(['De-id']).count().reset_index('De-id')
    df_login.columns = ['De-id', PRE_FIX + 'LOGIN_ATTEMPT']

    df_se_out = df_evt[df_evt['event_type'] == 'SESSION_TIMEOUT'].groupby(['De-id']).count().reset_index('De-id')
    df_se_out.columns = ['De-id', PRE_FIX + 'SESSION_TIMEOUT']

    df_logout = df_evt[df_evt['event_type'] == 'LOGOUT'].groupby(['De-id']).count().reset_index('De-id')
    df_logout.columns = ['De-id', PRE_FIX + 'LOGOUT']

    df_all = df_login
    df_all = pd.merge(df_all, df_se_out, on='De-id', how='left')
    df_all = pd.merge(df_all, df_logout, on='De-id', how='left')

    df_int_handle = df_t[['De-id', 'internal_handle']]

    group_list        = ['groups', 'cp_group_create_self_groupmem', 'group_file', 'group_file', 'group_forum', 'groups_sign_up', 'agroup', 'group_blogs','group_task_create', 'group_task_view','cp_group_edit_self_groupmem','group_file_add', 'group_email', 'cp_groups', 'cp_groups_settings','edit_group_blog_entry', 'db_forum_collection_group', 'group_tasks', 'group_journal','group_virtual_classroom', 'add_group_journal_entry','email_all_groups', 'edit_group_journal_entry', 'email_select_groups', 'add_group_blog_entry']
    db_list           = ['discussion_board_entry', 'db_thread_list_entry', 'discussion_board', 'db_thread_list','db_collection', 'db_collection_group', 'db_collection_entry', 'db_thread_list_group']
    myinfo_list       = ['my_inst_personal_info', 'my_inst_personal_settings','my_inst_personal_edit', 'my_inst_myplaces_settings','my_tasks', 'my_task_create', 'my_email_courses','my_task_view', 'my_announcements']
    course_list       = ['course_tools_area', 'course_task_view', 'enroll_course', 'classic_course_catalog']
    journal_list      = ['journal', 'journal_view', 'view_draft_journal_entry',  'add_journal_entry', 'edit_journal_entry']
    email_list        = ['send_email', 'email_all_instructors', 'email_all_students', 'email_select_students','email_all_users',  'email_select_groups','email_all_groups']
    staff_list        = ['staff_information', 'cp_staff_information']
    annoucements_list = ['my_announcements', 'announcements_entry', 'announcements', 'cp_announcements']
    content_list      = ['content', 'cp_content']
    grade_list        = ['check_grade']

    df_group        = int_handle_cnt(group_list, df_int_handle, 'group', PRE_FIX)
    df_db           = int_handle_cnt(db_list, df_int_handle, 'db', PRE_FIX)
    df_myinfo       = int_handle_cnt(myinfo_list, df_int_handle, 'myinfo', PRE_FIX)
    df_course       = int_handle_cnt(course_list, df_int_handle, 'course', PRE_FIX)
    df_journal      = int_handle_cnt(journal_list, df_int_handle, 'journal', PRE_FIX)
    df_email        = int_handle_cnt(email_list, df_int_handle, 'email', PRE_FIX)
    df_staff        = int_handle_cnt(staff_list, df_int_handle, 'staff', PRE_FIX)
    df_annoucements = int_handle_cnt(annoucements_list, df_int_handle, 'annoucements', PRE_FIX)
    df_content      = int_handle_cnt(content_list, df_int_handle, 'content', PRE_FIX)
    df_grade        = int_handle_cnt(grade_list, df_int_handle, 'grade', PRE_FIX)

    dfs = [df_group, df_db, df_myinfo, df_course, df_journal, df_email, df_staff, df_annoucements, df_content, df_grade]

    for df in dfs:
        df_all = pd.merge(df_all, df, on='De-id', how='left')   

    df_all = df_all.rename(columns={'De-id':'MASKED_STUDENT_ID'})
    return df_all

In [13]:
lib_se1 = pd.read_csv('Std_Lib_features_2016_se1.csv')
df_se1  = lib_se1
df_se1.head()    

,MASKED_STUDENT_ID,workday,weekend,morning,afternoon,evening,overnight,workday_ExamMonth,weekend_ExamMonth,morning_ExamMonth,...,weekend_firstMonth,morning_firstMonth,afternoon_firstMonth,evening_firstMonth,overnight_firstMonth,examMonth,notExamMonth,firstMonth,total_checkin,label_atRist
0,8TMIKVZ5,25,0,0,25,0,0,0,0,0,...,0,0,12,0,0,0,25,12,25,0
1,N2YYKTMZ,61,3,1,45,18,0,5,0,0,...,0,0,11,2,0,5,59,13,64,0
2,BZRW4GD3,0,1,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
3,HJTBF62Q,14,21,1,26,8,0,4,7,1,...,0,0,1,0,0,11,24,1,35,0
4,33XUIDIG,20,5,0,16,9,0,3,0,0,...,0,0,0,3,0,3,22,3,25,0


In [22]:
from datetime import datetime
start = datetime.now()

df = pd.read_csv('DR0008_activity_accumulator_2016_09.csv', sep='\t')
df = df[['De-id', 'event_type', 'course_id', 'internal_handle', 'timestamp']]
df['timestamp'] = pd.to_datetime(df['timestamp'])

current = datetime.now()
print(current-start)

df['first_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=1 and x.day <=7 else 0)
df['second_week'] = df['timestamp'].apply(lambda x: 1 if x.day >=8 and x.day <=15 else 0)
df['third_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=16 and x.day <=23 else 0)
df['forth_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=24 and x.day <=32 else 0)

df_first_week  = df[df['first_week'] == 1]
df_second_week = df[df['second_week'] == 1]
df_third_week  = df[df['third_week'] == 1]
df_forth_week  = df[df['forth_week'] == 1]

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_first_week, 'first_week')
df_se1 = pd.merge(df_se1, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_second_week, 'second_week')
df_se1 = pd.merge(df_se1, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_third_week, 'third_week')
df_se1 = pd.merge(df_se1, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_forth_week, 'forth_week')
df_se1 = pd.merge(df_se1, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

0:00:40.716781
0:05:54.778592
0:05:56.991178
0:06:00.471030
0:06:03.717643
0:06:06.892500


In [25]:
start = datetime.now()

df = pd.read_csv('DR0008_activity_accumulator_2016-10.csv', sep='\t')
df = df[['De-id', 'event_type', 'course_id', 'internal_handle', 'timestamp']]
df['timestamp'] = pd.to_datetime(df['timestamp'])

current = datetime.now()
print(current-start)

df['fifth_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=1 and x.day <=7 else 0)
df['sixth_week'] = df['timestamp'].apply(lambda x: 1 if x.day >=8 and x.day <=15 else 0)
df['seventh_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=16 and x.day <=23 else 0)
df['eighth_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=24 and x.day <=32 else 0)

df_fifth_week  = df[df['fifth_week'] == 1]
df_sixth_week = df[df['sixth_week'] == 1]
df_seventh_week  = df[df['seventh_week'] == 1]
df_eighth_week  = df[df['eighth_week'] == 1]

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_fifth_week, 'fifth_week')
df_se1 = pd.merge(df_se1, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_sixth_week, 'sixth_week')
df_se1 = pd.merge(df_se1, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_seventh_week, 'seventh_week')
df_se1 = pd.merge(df_se1, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_eighth_week, 'eighth_week')
df_se1 = pd.merge(df_se1, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

## ************************************

df = pd.read_csv('DR0008_activity_accumulator_2016-11.csv', sep='\t')
df = df[['De-id', 'event_type', 'course_id', 'internal_handle', 'timestamp']]
df['timestamp'] = pd.to_datetime(df['timestamp'])

current = datetime.now()
print(current-start)

df['nineth_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=1 and x.day <=7 else 0)
df['tenth_week'] = df['timestamp'].apply(lambda x: 1 if x.day >=8 and x.day <=15 else 0)
df['eleventh_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=16 and x.day <=23 else 0)
df['twelfth_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=24 and x.day <=32 else 0)

df_nineth_week  = df[df['nineth_week'] == 1]
df_tenth_week = df[df['tenth_week'] == 1]
df_eleventh_week  = df[df['eleventh_week'] == 1]
df_twelfth_week  = df[df['twelfth_week'] == 1]

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_nineth_week, 'nineth_week')
df_se1 = pd.merge(df_se1, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_tenth_week, 'tenth_week')
df_se1 = pd.merge(df_se1, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_eleventh_week, 'eleventh_week')
df_se1 = pd.merge(df_se1, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_twelfth_week, 'twelfth_week')
df_se1 = pd.merge(df_se1, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df = pd.read_csv('DR0008_activity_accumulator_2016-12.csv', sep='\t')
df = df[['De-id', 'event_type', 'course_id', 'internal_handle', 'timestamp']]
df['timestamp'] = pd.to_datetime(df['timestamp'])

current = datetime.now()
print(current-start)

df['thirteenth_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=1 and x.day <=7 else 0)
df['forteenth_week'] = df['timestamp'].apply(lambda x: 1 if x.day >=8 and x.day <=15 else 0)

df_thirteenth_week  = df[df['thirteenth_week'] == 1]
df_forteenth_week = df[df['forteenth_week'] == 1]

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_thirteenth_week, 'thirteenth_week')
df_se1 = pd.merge(df_se1, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_forteenth_week, 'forteenth_week')
df_se1 = pd.merge(df_se1, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)


0:00:48.099530
0:06:54.699618
0:06:57.585644
0:07:00.321024
0:07:03.574323
0:07:06.797592
0:07:59.669791
0:14:42.114957
0:14:44.930554
0:14:48.118347
0:14:51.447041
0:14:54.565380
0:15:24.637065
0:17:26.781120
0:17:30.115091
0:17:32.680123
0:17:32.680336


In [28]:
df_se1.to_csv('2016_se1_stat_features.csv')

In [29]:
lib_se2 = pd.read_csv('Std_Lib_features_2016_se2.csv')
df_se2  = lib_se2
df_se2.head()  

start = datetime.now()

df = pd.read_csv('DR0008_activity_accumulator_2017-02.csv', sep='\t')
df = df[['De-id', 'event_type', 'course_id', 'internal_handle', 'timestamp']]
df['timestamp'] = pd.to_datetime(df['timestamp'])

current = datetime.now()
print(current-start)

df['first_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=1 and x.day <=7 else 0)
df['second_week'] = df['timestamp'].apply(lambda x: 1 if x.day >=8 and x.day <=15 else 0)
df['third_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=16 and x.day <=23 else 0)
df['forth_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=24 and x.day <=32 else 0)

df_first_week  = df[df['first_week'] == 1]
df_second_week = df[df['second_week'] == 1]
df_third_week  = df[df['third_week'] == 1]
df_forth_week  = df[df['forth_week'] == 1]

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_first_week, 'first_week')
df_se2 = pd.merge(df_se2, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_second_week, 'second_week')
df_se2 = pd.merge(df_se2, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_third_week, 'third_week')
df_se2 = pd.merge(df_se2, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_forth_week, 'forth_week')
df_se2 = pd.merge(df_se2, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)


df = pd.read_csv('DR0008_activity_accumulator_2017-03.csv', sep='\t')
df = df[['De-id', 'event_type', 'course_id', 'internal_handle', 'timestamp']]
df['timestamp'] = pd.to_datetime(df['timestamp'])

current = datetime.now()
print(current-start)

df['fifth_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=1 and x.day <=7 else 0)
df['sixth_week'] = df['timestamp'].apply(lambda x: 1 if x.day >=8 and x.day <=15 else 0)
df['seventh_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=16 and x.day <=23 else 0)
df['eighth_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=24 and x.day <=32 else 0)

df_fifth_week  = df[df['fifth_week'] == 1]
df_sixth_week = df[df['sixth_week'] == 1]
df_seventh_week  = df[df['seventh_week'] == 1]
df_eighth_week  = df[df['eighth_week'] == 1]

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_fifth_week, 'fifth_week')
df_se2 = pd.merge(df_se2, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_sixth_week, 'sixth_week')
df_se2 = pd.merge(df_se2, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_seventh_week, 'seventh_week')
df_se2 = pd.merge(df_se2, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_eighth_week, 'eighth_week')
df_se2 = pd.merge(df_se2, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

## ************************************

df = pd.read_csv('DR0008_activity_accumulator_2017-04.csv', sep='\t')
df = df[['De-id', 'event_type', 'course_id', 'internal_handle', 'timestamp']]
df['timestamp'] = pd.to_datetime(df['timestamp'])

current = datetime.now()
print(current-start)

df['nineth_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=1 and x.day <=7 else 0)
df['tenth_week'] = df['timestamp'].apply(lambda x: 1 if x.day >=8 and x.day <=15 else 0)
df['eleventh_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=16 and x.day <=23 else 0)
df['twelfth_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=24 and x.day <=32 else 0)

df_nineth_week  = df[df['nineth_week'] == 1]
df_tenth_week = df[df['tenth_week'] == 1]
df_eleventh_week  = df[df['eleventh_week'] == 1]
df_twelfth_week  = df[df['twelfth_week'] == 1]

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_nineth_week, 'nineth_week')
df_se2 = pd.merge(df_se2, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_tenth_week, 'tenth_week')
df_se2 = pd.merge(df_se2, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_eleventh_week, 'eleventh_week')
df_se2 = pd.merge(df_se2, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_twelfth_week, 'twelfth_week')
df_se2 = pd.merge(df_se2, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df = pd.read_csv('DR0008_activity_accumulator_2017-05.csv', sep='\t')
df = df[['De-id', 'event_type', 'course_id', 'internal_handle', 'timestamp']]
df['timestamp'] = pd.to_datetime(df['timestamp'])

current = datetime.now()
print(current-start)

df['thirteenth_week']  = df['timestamp'].apply(lambda x: 1 if x.day >=1 and x.day <=7 else 0)
df['forteenth_week'] = df['timestamp'].apply(lambda x: 1 if x.day >=8 and x.day <=15 else 0)

df_thirteenth_week  = df[df['thirteenth_week'] == 1]
df_forteenth_week = df[df['forteenth_week'] == 1]

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_thirteenth_week, 'thirteenth_week')
df_se2 = pd.merge(df_se2, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

df_tmp = extract_functions(df_forteenth_week, 'forteenth_week')
df_se2 = pd.merge(df_se2, df_tmp, on = ['MASKED_STUDENT_ID'], how='left').fillna(0)

current = datetime.now()
print(current-start)

0:00:28.670707
0:04:18.593330
0:04:20.101380
0:04:22.501756
0:04:25.039097
0:04:26.638891
0:05:06.266615
0:10:17.105125
0:10:19.422698
0:10:22.164377
0:10:24.874090
0:10:27.425198
0:11:05.387788
0:15:59.654940
0:16:01.796332
0:16:04.215670
0:16:06.974428
0:16:09.130141
0:16:23.550487
0:17:20.106447
0:17:21.645787
0:17:23.023510


In [30]:
df_se2.to_csv('2016_se2_stat_features.csv')